### Ultrasonic sensor test example ###

In [3]:
from pynq import Overlay, MMIO, PL, allocate
from pynq.lib.video import *
from pynq_dpu import DpuOverlay
import cv2
import numpy as np
import time
import os
import spidev
import keyboard
from driving_system_controller import DrivingSystemController
from image_processor import ImageProcessor
from config import MOTOR_ADDRESSES, ADDRESS_RANGE,classes_path, anchors
from AutoLab_lib import init

init()
# Initialize SPI
spi0 = spidev.SpiDev()
spi0.open(0, 0)
spi0.max_speed_hz = 20000000
spi0.mode = 0b00

# 자율주행 모드 뒷바퀴 & 조향 속도 설정 (0 ~ 100)
speed = 50
steering_speed = 50
motors = {}

for name, addr in MOTOR_ADDRESSES.items():
    motors[name] = MMIO(addr, ADDRESS_RANGE)


def load_dpu():
    global dpu, input_data, output_data, shapeIn, shapeOut0, shapeOut1
    
    overlay = DpuOverlay("../dpu/dpu.bit")
    overlay.load_model("../xmodel/tiny-yolov3_coco_256.xmodel")
    
    dpu = overlay.runner
    print("[DEBUG] DPU runner:", dpu)  # None이 아니어야 정상
    
    return overlay, dpu

def main():
    overlay = load_dpu()
    controller = DrivingSystemController(overlay, dpu, motors, speed, steering_speed)
    controller.run(camera_index=0)

# if __name__ == "__main__":
#     main()

IndentationError: expected an indented block after 'if' statement on line 137 (motor_controller.py, line 138)

In [ ]:
overlay = load_dpu()

In [ ]:
# motor_controller 없이 DrivingSystemController 생성해보기
# (임시 dummy motors dict 사용)
controller = DrivingSystemController(overlay, dpu, {}, speed, steering_speed)
print("DrivingSystemController 생성 OK")

In [ ]:
from pynq import MMIO
from config import MOTOR_ADDRESSES, ADDRESS_RANGE

motors = {}

for name, addr in MOTOR_ADDRESSES.items():
    try:
        print(f"[MMIO 초기화 시도] {name} → addr = {hex(addr)}")
        motors[name] = MMIO(addr, ADDRESS_RANGE)
        print(f"  ✅ 성공적으로 접근: {name}")
    except Exception as e:
        print(f"  ❌ MMIO 접근 실패: {name}, 에러: {e}")


In [ ]:
from pynq import MMIO
from config import MOTOR_ADDRESSES, ADDRESS_RANGE

print("MMIO 접근 테스트 시작")

for name, addr in MOTOR_ADDRESSES.items():
    print(f"\n[{name}] 주소: {hex(addr)}")
    try:
        m = MMIO(addr, ADDRESS_RANGE)
        print("  ✅ MMIO 인스턴스 생성 성공")

        print("  [WRITE] 0x00 ← 600600")
        m.write(0x00, 600600)

        print("  [WRITE] 0x04 ← 300300")
        m.write(0x04, 300300)

        print("  [WRITE] 0x08 ← 0")
        m.write(0x08, 0)

        print("  ✅ Register write 성공")
    except Exception as e:
        print(f"  ❌ 에러 발생: {e}")


In [ ]:
for name, addr in MOTOR_ADDRESSES.items():
    print(f"테스트 중: {name} - {hex(addr)}")
    m = MMIO(addr, ADDRESS_RANGE)

    if name == 'motor_0': continue  # 하나씩 활성화하며 테스트
    if name == 'motor_1': continue

    print("  write(0x00, ...)")
    m.write(0x00, 600600)  # 여기서 죽으면 해당 motor가 문제

    print("  write(0x04, ...)")
    m.write(0x04, 300300)

    print("  write(0x08, ...)")
    m.write(0x08, 0)


In [ ]:

print("[2] DrivingSystemController 생성 시작")
controller = DrivingSystemController(overlay, dpu, motors, speed, steering_speed)
print("[3] DrivingSystemController 생성 완료")


In [ ]:

print("[4] run() 함수 진입 전")
controller.run()